In [123]:
# import dependencies
from __future__ import print_function
import numpy as np
import mxnet as mx
import mxnet.ndarray as F
from mxnet import gluon, autograd
from mxnet.gluon import nn
import logging
logging.getLogger().setLevel(logging.INFO)

In [201]:
context_size = 2
embeding_dim = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
vocab_size = len(vocab)
print(vocab_size, len(trigrams))

word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
97 113


In [135]:
class Net(nn.Block):
    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            self.embed = nn.Embedding(vocab_size, embeding_dim)
            self.fc1 = nn.Dense(128, in_units= embeding_dim * context_size)
            self.fc2 = nn.Dense(vocab_size)

    def forward(self, x):
        x = self.embed(x)
        # 0 means copy over size from corresponding dimension.
        # -1 means infer size from the rest of dimensions.
        x = x.reshape((1, -1))
        out = F.relu(self.fc1(x))
        out = self.fc2(out)
        #out = F.log_softmax(out)
        return out
        

In [242]:
ctx = mx.cpu(0)
net = Net()
net.collect_params().initialize(mx.init.Uniform(0.31), ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.001})


In [243]:

# print(net.collect_params().keys())
# print("net paramss", net.collect_params()['net80_embedding0_weight'].set_data(nd.array(torch)))
# print(net.fc2.weight.data())
#metric = mx.metric.Accuracy()
loss = gluon.loss.SoftmaxCrossEntropyLoss()

losses = []
for epoch in range(10):
    total_loss = 0
    #metric.reset()
    for context, target in trigrams:     
        context_idxs = [word_to_ix[w] for w in context]
        context_var = mx.nd.array(context_idxs)
        label = mx.nd.array([word_to_ix[target]])
        with autograd.record():       
                net.params.zero_grad
                log_probs = net(context_var)
                L = loss(log_probs, label)
                L.backward()
        
        trainer.step(1)
    #    metric.update([label], [log_probs])
        total_loss += mx.nd.array(L).asscalar()  
    losses.append(total_loss)
    
#     name, acc = metric.get()
#     print('[Epoch %d] Training: %s=%f'%(epoch, name, acc))

print(losses)

[517.26326894760132, 516.66081476211548, 516.06235694885254, 515.46494150161743, 514.86982488632202, 514.28023815155029, 513.69159698486328, 513.10643243789673, 512.52707481384277, 511.95334434509277]
